In [48]:
# This is the template for the submission. If you want, you can develop your algorithm in a regular Python script and copy the code here for submission.

# Team members (e-mail, legi):
# zhisun@ethz.ch, 22-958-227
# enjcao@ethz.ch, 22-942-700
# yifzhou@ethz.ch, 22-940-381

In [2]:
import pandas as pd
import math
from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset
from os import listdir
from os.path import isfile, join
from math import sqrt
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Get the path of all traces
dir_data = 'E:\\Sunzhichao\\ETHz\\2223Spring\\Mobile_Health\\data\\'
dir_traces = dir_data + 'train\\'
dir_labels = dir_data + 'labels\\'
dir_loaded = dir_data + 'Loaded_data\\'
dir_models = dir_data + 'models\\'
# recorded
dir_recorded = 'data/recorded'

In [7]:
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f)) and f[-5:] == '.json']
filenames.sort()

In [19]:
# load magnetometer data and path index ground truth
data_magnet = np.load(dir_loaded + "train_data_magnet.npy", allow_pickle=True).item()
data_acc = np.load(dir_loaded + "train_data_acc_resample.npy", allow_pickle=True).item()
path_indexs = np.load(dir_loaded + "train_path_label_corrected.npy")
data_len = np.shape(data_magnet['magn'])[1]

In [6]:
# compute the change of data
def compute_rate_of_change(data):
    rate_of_change = np.diff(data)
    return rate_of_change

Following blocks are CNN model for magnatometer data only(magnitute or x, y, z data)

In [43]:
# Define the CNN model for x, y, z data
class CNN1D_3(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D_3, self).__init__()
        self.conv1 = nn.Conv1d(3, 32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 247, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [7]:
# Define the CNN model for magnitute data
class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(p=0.5)  # Add dropout layer with 50% dropout probability
        self.fc1 = nn.Linear(64 * 247, 255)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(255, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.dropout(x)  # Apply dropout
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [8]:
# Define a custom dataset class
class PathDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
# Define the L1 regularization term
def l1_regularization(model, lambda_l1):
    l1_loss = torch.tensor(0., requires_grad=True)
    for param in model.parameters():
        l1_loss = torch.norm(param, 1) + l1_loss
    return lambda_l1 * l1_loss

In [33]:
# Convert resampled data and labels to tensors
X = torch.tensor(np.expand_dims(data_magnet['magn'], axis=-1), dtype=torch.float32)
# all_axes_data = np.stack((data_magnet['x'], data_magnet['y'], data_magnet['z']), axis=-1)
# X = torch.tensor(all_axes_data, dtype=torch.float32)
y = torch.tensor(path_indexs, dtype=torch.long)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create dataset and dataloader objects for training and validation sets
train_dataset = PathDataset(X_train, y_train)
val_dataset = PathDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the CNN model and other training components
num_classes = 5
model = CNN1D(num_classes).to(device)
# Set the L1 regularization strength (lambda_l1)
lambda_l1 = 1e-5

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training parameters
num_epochs = 300
best_val_accuracy = 0

# Training loop
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    correct_train = 0
    total_train = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.permute(0, 2, 1)
        optimizer.zero_grad()
        outputs = model(inputs)
        # Compute the number of correct predictions and the total number of predictions
        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()
        # Compute the total loss, including the L1 regularization term
        loss = criterion(outputs, targets) + l1_regularization(model, lambda_l1)
        
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct_train / total_train
    
    # Validation loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            inputs = inputs.permute(0, 2, 1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    val_accuracy = correct / total

    # Save the model with the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), dir_models+"path_index_cnn_model.pth")

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f},Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

print(f"Best validation accuracy: {best_val_accuracy:.4f}")

NameError: name 'PathDataset' is not defined

In [81]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold


# Convert resampled data and labels to tensors
data = np.transpose(np.expand_dims(data_magnet['magn'], axis=-1), (0, 2, 1))
data = torch.tensor(data, dtype=torch.float32)
# all_axes_data = np.stack((data_magnet['x'], data_magnet['y'], data_magnet['z']), axis=-1)
# X = torch.tensor(all_axes_data, dtype=torch.float32)
labels = torch.tensor(path_indexs, dtype=torch.long)

# Define hyperparameters for grid search
learning_rates = [0.00001, 0.000005, 0.000001]
num_epochs = 100
batch_sizes = [16, 32, 64]
weight_decays = [0, 1e-4, 1e-5]

# Define the number of folds for cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True)

# Function to train and evaluate the model
def train_and_evaluate(model, train_loader, val_loader, optimizer, criterion):
    best_val_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        val_accuracy = correct / total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
        # print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f},Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

    return best_val_accuracy

# Perform grid search and k-fold cross-validation
best_hyperparams = None
best_cross_val_accuracy = 0
for lr in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            fold_accuracies = []
            for train_indices, val_indices in kf.split(data):
                # Create the model
                num_classes = 5
                model = CNN1D(num_classes).to(device)
                
                # Create the optimizer with the current hyperparameters
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
                
                # Define the loss function (criterion)
                criterion = nn.CrossEntropyLoss()

                # Prepare the data loaders for the current fold
                train_dataset = PathDataset(data[train_indices], labels[train_indices])
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_dataset = PathDataset(data[val_indices], labels[val_indices])
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                # Set device
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                
                
                # Train and evaluate the model with the current fold and hyperparameters
                fold_accuracy = train_and_evaluate(model, train_loader, val_loader, optimizer, criterion)
                fold_accuracies.append(fold_accuracy)

            # Calculate the average accuracy across all folds for the current hyperparameters
            cross_val_accuracy = np.mean(fold_accuracies)
            print(f'lr={lr}, batch_size={batch_size}, weight_decay={weight_decay}, cross_val_accuracy={cross_val_accuracy}')

            # Update the best hyperparameters if the current cross_val_accuracy is higher
            if cross_val_accuracy > best_cross_val_accuracy:
                best_cross_val_accuracy = cross_val_accuracy
                best_hyperparams = {'lr': lr, 'batch_size': batch_size, 'weight_decay': weight_decay}

print(f'Best hyperparameters: {best_hyperparams}')
print(f'Best cross-validation accuracy: {best_cross_val_accuracy}')
               


lr=1e-05, batch_size=16, weight_decay=0, cross_val_accuracy=0.5185185185185185
lr=1e-05, batch_size=16, weight_decay=0.0001, cross_val_accuracy=0.5074074074074074
lr=1e-05, batch_size=16, weight_decay=1e-05, cross_val_accuracy=0.5148148148148148
lr=1e-05, batch_size=32, weight_decay=0, cross_val_accuracy=0.47407407407407404
lr=1e-05, batch_size=32, weight_decay=0.0001, cross_val_accuracy=0.4888888888888888
lr=1e-05, batch_size=32, weight_decay=1e-05, cross_val_accuracy=0.5111111111111111
lr=1e-05, batch_size=64, weight_decay=0, cross_val_accuracy=0.49629629629629635
lr=1e-05, batch_size=64, weight_decay=0.0001, cross_val_accuracy=0.4814814814814815
lr=1e-05, batch_size=64, weight_decay=1e-05, cross_val_accuracy=0.4814814814814815
lr=5e-06, batch_size=16, weight_decay=0, cross_val_accuracy=0.5
lr=5e-06, batch_size=16, weight_decay=0.0001, cross_val_accuracy=0.5074074074074074
lr=5e-06, batch_size=16, weight_decay=1e-05, cross_val_accuracy=0.5185185185185185
lr=5e-06, batch_size=32, weig

Following blocks are for MLP model for accelerometer data and magnatometer data

In [47]:
# MLP for accelerometer and magnetometer data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_length, num_sensors, num_classes):
        super(MLP, self).__init__()
        self.input_length = input_length
        self.num_sensors = num_sensors
        # self.fc1 = nn.Linear(input_length * num_sensors, hidden_size)
        # self.relu = nn.ReLU()
        # self.fc2 = nn.Linear(hidden_size, num_classes)
        # self.sigmoid = torch.nn.Sigmoid()

        self.fc = torch.nn.Sequential(
        torch.nn.Linear(in_features=input_length * num_sensors, out_features=2000),
        # torch.nn.ReLU(),
        # torch.nn.Dropout(0.5),
        # torch.nn.Linear(in_features=2000, out_features=1000),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(in_features=2000, out_features=num_classes),
        torch.nn.Sigmoid())

    def forward(self, x):
        x = x.view(-1, self.input_length * self.num_sensors)
        # x = self.fc1(x)
        # x = self.relu(x)
        # x = self.fc2(x)
        # x = self.sigmoid(x)
        x = self.fc(x)
        return x

In [51]:

# Convert resampled data and labels to tensors
batch_size = 16
all_axes_data = np.stack((data_magnet['x'], data_magnet['y'], data_magnet['z'], data_acc['x'], data_acc['y'], data_acc['z']), axis=-1)
X = torch.tensor(all_axes_data, dtype=torch.float32)
y = torch.tensor(path_indexs, dtype=torch.long)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

# Create DataLoader objects for efficient batching
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Set the input size, hidden size, and number of classes
input_size = X_train.shape[1]
num_classes = len(torch.unique(labels))
num_sensors = 6
# Instantiate the model, loss function, and optimizer
model = MLP(input_size, num_sensors, num_classes)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-06)
optimizer = optim.Adam(model.parameters(), lr=5e-07)
# Set seed to make result reproducible
torch.manual_seed(0)

# Train the model
num_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    correct_train = 0
    total_train = 0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        
        outputs = model(inputs)
        # Compute the number of correct predictions and the total number of predictions
        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct_train / total_train

    # Evaluate on validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f},Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")



Epoch 1/500, Loss: 1.6910,Train Accuracy: 18.06%, Validation Accuracy: 27.78%
Epoch 2/500, Loss: 1.6745,Train Accuracy: 23.15%, Validation Accuracy: 25.93%
Epoch 3/500, Loss: 1.6940,Train Accuracy: 18.06%, Validation Accuracy: 25.93%
Epoch 4/500, Loss: 1.7128,Train Accuracy: 16.67%, Validation Accuracy: 25.93%
Epoch 5/500, Loss: 1.6481,Train Accuracy: 24.07%, Validation Accuracy: 25.93%
Epoch 6/500, Loss: 1.6587,Train Accuracy: 18.06%, Validation Accuracy: 24.07%
Epoch 7/500, Loss: 1.6843,Train Accuracy: 16.67%, Validation Accuracy: 24.07%
Epoch 8/500, Loss: 1.6420,Train Accuracy: 23.15%, Validation Accuracy: 24.07%
Epoch 9/500, Loss: 1.6638,Train Accuracy: 23.15%, Validation Accuracy: 24.07%
Epoch 10/500, Loss: 1.6371,Train Accuracy: 22.69%, Validation Accuracy: 25.93%
Epoch 11/500, Loss: 1.6741,Train Accuracy: 20.37%, Validation Accuracy: 25.93%
Epoch 12/500, Loss: 1.6448,Train Accuracy: 24.07%, Validation Accuracy: 27.78%
Epoch 13/500, Loss: 1.6593,Train Accuracy: 24.54%, Validation

Following blocks are for LSTM model for accelerometer data and magnatometer data

In [ ]:
import torch

class LSTM(torch.nn.Module):
    """
    Simple RNN with LSTM cells and linear projection head.
    """
    def __init__(self, batch_size, input_size, hidden_size, num_layers, output_size, device):
        """
        Constructor for RNN_LSTM.
        :param hidden_size: The number of units for each LSTM cell.
        :param num_layers: The number of LSTM cells we want to use.
        :param input_size: The size of the text vocabulary.
        :param device: The device we are using to run the model.
        """   
        super(LSTM, self).__init__()
        # LSTM module
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers)
        # Fully connected layer - The output tensor of the lstm has shape [seq_len, batch, hidden_size],
        # i.e. it contains the outputs of the last cell for every time step.
        # We want to map the output back to the "vocabulary space", so we add a linear layer.
        # Importantly, the linear layer should share its parameters across time steps.
        # This is the case in PyTorch where the linear layer is only applied to the last dimenesion.
        self.fc = torch.nn.Linear(hidden_size, output_size)
        # Sigmoid layer
        self.sigmoid = torch.nn.Sigmoid()
        # Embedding layer TODO: (no need for embedding here?)
        # self.embedding = torch.nn.Embedding(input_size, input_size)
        # self.embedding.weight.data = torch.eye(input_size)

        # Variables 
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
        # Set initial cell and hidden states to zero
        self.reset_hidden_state(batch_size)

    def reset_hidden_state(self, batch_size):
        """
        Reset initial cell and hidden states to zero. The batch size might be different during training
        and inference. The parameter allows to modify it consequently. 
        :param batch_size: batch size used on the model.
        """
        self.h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        self.c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)

    def forward(self, x):
        """
        Compute the forward pass of the network.
        :param x: The input tensor of size (batch, seq_len).
        :return: The activated output of the network. 
        """
        # x = self.one_hot(x) # (batch, seq_len, input_size)
        x = x.transpose(0, 1) # (seq_len, batch, input_size)
        x, (self.h_0, self.c_0) = self.lstm(x, (self.h_0, self.c_0)) #  (seq_len, batch, hidden_size)
        x = self.fc(x)  #  (seq_len, batch_size, output_size)
        return x

In [ ]:
def train(data_loader, device, n_epochs, use_features=False):
    """
    Train model for n_epochs.
    """
    
    model.train()
    for epoch in range(n_epochs):    
        total_loss, total_accuracy = 0., 0.
        for i, (x1, x2, y) in enumerate(data_loader):
            # x = x2
            x = torch.cat((x1, x2), dim=2)
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()   # zero the parameter gradients   
            model.reset_hidden_state(batch_size)    # reset the hidden states as we are training on a new sequence
            y_hat = model(x)[-1, :, :]   # forward pass
            loss = cross_entropy_loss(y_hat, y)  # compute loss
            loss.backward() # backward pass
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)  # clip the gradient to prevent exploding gradient
            optimizer.step()    # perform update 
            total_loss += loss.item()
            total_accuracy += (torch.argmax(y_hat, dim=1) == y).float().sum()

        total_loss /= len(data_loader)
        total_accuracy /= len(data_loader)*batch_size
        print(f"[Epoch {epoch}] - Training : accuracy = {total_accuracy}, loss = {total_loss}")
    return total_loss, total_accuracy.item()

def val(data_loader, device, use_features=False):
    model.eval()
    with torch.no_grad():
        right_cases = 0
        all_cases = 0
        
        for i, (x1, x2, y) in enumerate(data_loader):
            # x = x2
            x = torch.cat((x1, x2), dim=2)
            x = x.to(device)
            y = y.to(device)
            model.reset_hidden_state(batch_size)
            y_hat = model(x)[-1, :, :]
            right_cases += np.sum((torch.argmax(y_hat, dim=1) == y).cpu().numpy())
            all_cases += batch_size
    accuracy = right_cases / all_cases
    print(f"validation accuracy = {accuracy}")
    return accuracy

def train_val_save(train_data_loader, val_data_loader, device, n_epochs, partition_epoch, use_features=False):
    global model
    loop_num = int(n_epochs / partition_epoch)
    epochs = 0
    max_val_accuracy = 0
    train_loss_list, train_accuracy_list, validation_accuracy_list = [], [], []
    for _ in range(loop_num):
        train_loss, train_acc = train(train_data_loader, device, partition_epoch, use_features)
        train_loss_list += [train_loss]
        train_accuracy_list += [train_acc]
        epochs += partition_epoch
        accuracy = val(val_data_loader, device, use_features)
        validation_accuracy_list += [accuracy]
        if accuracy > max_val_accuracy:
            max_val_accuracy = accuracy
            print(f"\nAt epoch {epochs}, max validation accuracy is updated to {accuracy}, model is saved.\n")
            model = model.to("cpu")
            torch.save(model, 'LSTM.pt')
            model = model.to("cuda")

    return train_loss_list, train_accuracy_list, validation_accuracy_list


if __name__ == '__main__':
    hidden_size = 256
    num_layers = 2
    batch_size = 16
    output_size = 36

    # Select device on which we will train our model 
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Convert resampled data and labels to tensors
    batch_size = 16
    all_axes_data = np.stack((data_magnet['x'], data_magnet['y'], data_magnet['z'], data_acc['x'], data_acc['y'], data_acc['z']), axis=-1)
    X = torch.tensor(all_axes_data, dtype=torch.float32)
    y = torch.tensor(path_indexs, dtype=torch.long)
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

    # Create DataLoader objects for efficient batching
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    seq_length = train_dataset[0][0].shape[0]

    if train_dataset.use_features:
        input_size = train_dataset[0][0].shape[1]
    else:
        # input_size = train_dataset[0][1].shape[1]
        input_size = train_dataset[0][0].shape[1] + train_dataset[0][1].shape[1]
    print(f'\nSequence length: {seq_length}. Input size: {input_size}\n')

    # build the model
    model = LSTM(batch_size, input_size, hidden_size, num_layers, output_size, device)
    model = model.to(device)
    cross_entropy_loss = torch.nn.CrossEntropyLoss()

    # count total number of parameters including non trainable
    total_params_count = sum(p.numel() for p in model.parameters())
    # count total trainable parameters
    trainable_params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total number of trainable parameters: {total_params_count}")
    print(f"Number of trainable parameters: {trainable_params_count}")

    # Optimization algorithm : Adam 
    learning_rate = 1e-4
    # weight_decay = 1e-5
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loss_list, train_accuracy_list, validation_accuracy_list = train_val_save(train_data_loader, val_data_loader, device, 150, 1, train_dataset.use_features)

    # save model on CPU 
    # model = model.to("cpu")

    # Save the model, the optimizer state and current number of epochs
    # torch.save(model, 'LSTM.pt')
    plt.plot(train_loss_list, label = "Train loss")
    plt.plot(train_accuracy_list, label = "Train acc")
    plt.plot(validation_accuracy_list, label = "Validation acc")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs Epoch')
    plt.legend()
    plt.savefig('lstm_loss.png')
